In [1]:
import sys
sys.path.append("../")
import numpy as np
import matplotlib.pyplot as plt
import scripts.prediction_utils as predict_u
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_plot_function as theodo_p
import scripts.groundtruth_utils as theodo_g
from numpy import linalg
import importlib
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)

In [2]:
def plotEllipsoid(A, center):
	"""Plot Ellipsoid with A matrix and center"""
	# random select color
	color = tuple(np.random.randint(256, size=3) / 255)

	# find the rotation matrix and radii of the axes
	U, s, rotation = linalg.svd(A)
	radii = 1.0 / np.sqrt(s)

	u = np.linspace(0, 2 * np.pi, 100)
	v = np.linspace(0, np.pi, 100)

	x = radii[0] * np.outer(np.cos(u), np.sin(v))
	y = radii[1] * np.outer(np.sin(u), np.sin(v))
	z = radii[2] * np.outer(np.ones_like(u), np.cos(v))

	for i, _ in enumerate(x):
		for j, _ in enumerate(x):
			[x[i, j], y[i, j], z[i, j]] = np.dot([x[i, j], y[i, j], z[i, j]], rotation) + center

	fig = plt.figure()
	ax = fig.add_subplot(111, projection='3d')
	ax.plot_wireframe(x, y, z, rstride=3, cstride=3, color=color, alpha=0.4)
	ax.scatter(center[0],center[1],center[2])
	ax.set_xlabel("X")
	ax.set_ylabel("Y")
	ax.set_zlabel("Z")
	plt.show()

def Ellipsoid_coordinates(A, center):
	# find the rotation matrix and radii of the axes
	U, s, rotation = linalg.svd(A)
	radii = 1.0 / np.sqrt(s)

	u = np.linspace(0, 2 * np.pi, 100)
	v = np.linspace(0, np.pi, 100)

	x = radii[0] * np.outer(np.cos(u), np.sin(v))
	y = radii[1] * np.outer(np.sin(u), np.sin(v))
	z = radii[2] * np.outer(np.ones_like(u), np.cos(v))

	for i, _ in enumerate(x):
		for j, _ in enumerate(x):
			[x[i, j], y[i, j], z[i, j]] = np.dot([x[i, j], y[i, j], z[i, j]], rotation) + center
	return x,y,z

def plot_three_Ellipsoids(x1,y1,z1,x2,y2,z2,x3,y3,z3,c1,c2,c3):

	fig = plt.figure()
	ax = fig.add_subplot(111, projection='3d')

	color = tuple(np.random.randint(256, size=3) / 255)
	ax.plot_wireframe(x1, y1, z1, rstride=3, cstride=3, color="r", alpha=0.25)
	ax.scatter(c1[0],c1[1],c1[2], color = "r")

	color = tuple(np.random.randint(256, size=3) / 255)
	ax.plot_wireframe(x2, y2, z2, rstride=3, cstride=3, color="g", alpha=0.25)
	ax.scatter(c2[0],c2[1],c2[2], color = "g")

	color = tuple(np.random.randint(256, size=3) / 255)
	ax.plot_wireframe(x3, y3, z3, rstride=3, cstride=3, color="b", alpha=0.25)
	ax.scatter(c3[0],c3[1],c3[2], color = "b")
	ax.set_xlabel("X [m]")
	ax.set_ylabel("Y [m]")
	ax.set_zlabel("Z [m]")
	plt.show()

In [3]:
class Ellipsoid:
	"""Class to make ellipsoid."""

	@staticmethod
	def getMinVolEllipse(P, tolerance=0.01):
		"""Find the minimum volume of ellipsoid which holds all the points.

		based on code from https://github.com/minillinim/ellipsoid and modified.

		Parameters:
			P: numpy array of N-dimensional points like this:
				P = [[x, y, z, ...], <--- one point per line
					 [x, y, z, ...],
					 [x, y, z, ...]]

			tolerance: convergence criterion

		Returns:
			(center, radii, rotation)
		"""
		N, d = np.shape(P)
		d = float(d)

		# Q will the working array
		Q = np.vstack([np.copy(P.T), np.ones(N)])
		QT = Q.T

		# initializations
		err = 1.0 + tolerance
		u = (1.0 / N) * np.ones(N)

		# Using Khachiyan Algorithm
		while err > tolerance:
			V = np.dot(Q, np.dot(np.diag(u), QT))
			M = np.diag(np.dot(QT, np.dot(linalg.inv(V), Q)))
			k = np.argmax(M)
			maximum = M[k]
			step_size = (maximum - d - 1.0) / ((d + 1.0) * (maximum - 1.0))
			u_n = (1 - step_size) * u
			u_n[k] += step_size
			err = linalg.norm(u_n - u)
			u = u_n

		# center of the ellipse
		center = np.dot(P.T, u)

		# A matrix for the ellipse
		A = linalg.inv(np.dot(P.T, np.dot(np.diag(u), P)) -
		               np.array([[a * b for b in center] for a in center])) / d

		# Get the radii and rotation
		U, s, rotation = linalg.svd(A)
		radii = 1 / np.sqrt(s)

		return center, radii, rotation

	@staticmethod
	def getEllipsoidVolume(radii):
		"""Calculate the volume of the blob"""
		return 4/3 * np.pi *radii[0]*radii[1]*radii[2]

	@staticmethod
	def plot_ellipsoid(center, radii, rotation, ax=None, plot_axes=False, showSurface=False, alpha=0.5):
		"""Plot an ellipsoid"""
		color = tuple(np.random.randint(256, size=3) / 255)

		make_ax = ax is None
		if make_ax:
			fig = plt.figure()
			ax = fig.add_subplot(111, projection='3d')

		u = np.linspace(0, 2*np.pi, 100)
		v = np.linspace(0, np.pi, 100)

		# get cartesian coordinates corresponding to the spherical angles:
		x = radii[0] * np.outer(np.cos(u), np.sin(v))
		y = radii[1] * np.outer(np.sin(u), np.sin(v))
		z = radii[2] * np.outer(np.ones_like(u), np.cos(v))

		# rotate accordingly
		for i, _ in enumerate(x):
			for j, _ in enumerate(x):
				[x[i, j], y[i, j], z[i, j]] = np.dot([x[i, j], y[i, j], z[i, j]], rotation) + center

		if plot_axes:
			axes = np.array([[radii[0], 0, 0],
			                 [0, radii[1], 0],
			                 [0, 0, radii[2]]])
			# rotate accordingly
			for i, axis in enumerate(axes):
				axes[i] = np.dot(axes[i], rotation)

			# plot axes
			for p in axes:
				X = np.linspace(-p[0], p[0], 100) + center[0]
				Y = np.linspace(-p[1], p[1], 100) + center[1]
				Z = np.linspace(-p[2], p[2], 100) + center[2]
				ax.plot(X, Y, Z, color=color)

		# Plot ellipsoid
		if showSurface:
			ax.plot_surface(x, y, z, rstride=3, cstride=3, color=color, alpha=alpha)
		else:
			ax.plot_wireframe(x, y, z, rstride=3, cstride=3, color=color, alpha=alpha)

		if make_ax:
			plt.show()

In [4]:
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)

P1 = np.array([0.5,0,0.5])
P2 = np.array([2,-1,0])
P3 = np.array([1,1,-1])

e12 = 0.02
e13 = 0.04
e23 = 0.06
ez = 0.002

C1, C2, C3 = predict_u.compute_covariance_prism(P1, P2, P3, e12, e13, e23, ez)

In [5]:
%matplotlib notebook

# 1r 2g 3b
# 12:2 13:4 23:6
x1,y1,z1 = Ellipsoid_coordinates(C1, P1)
x2,y2,z2 = Ellipsoid_coordinates(C2, P2)
x3,y3,z3 = Ellipsoid_coordinates(C3, P3)
plot_three_Ellipsoids(x1,y1,z1,x2,y2,z2,x3,y3,z3,P1,P2,P3)

<IPython.core.display.Javascript object>

# Process static experiments with point data

In [6]:
%matplotlib notebook
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)

# Important: set path to deployment data
path = "../data/20221103-2/"

# Important: change path to rosbag
file = "/home/maxime/data/FR2023/20221103/20221103_2_inter_prism.bag"

Tf = []
Tf.append(np.identity(4))
Tf.append(np.identity(4))
Tf.append(np.identity(4))
trajectory_trimble_1, trajectory_trimble_2, trajectory_trimble_3, time_trimble_1, time_trimble_2, time_trimble_3 = theodo_u.read_rosbag_theodolite_with_tf(file, Tf)
trimble_1 = np.array(trajectory_trimble_1).T
trimble_2 = np.array(trajectory_trimble_2).T
trimble_3 = np.array(trajectory_trimble_3).T

_, _, _, _, _, _, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data_pre_filtered(file)

/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)
/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)


Number of data for theodolites: [4896 4873 4962]
Bad measures: 1444
Number of data for theodolites: [4896 4873 4962]
Bad measures: 1444


In [44]:
%matplotlib notebook
plot_3d = 0   # Variable to plot in 3d, 0 = 2D, 1 = 3D
save_pdf = 0   # Variable to save file in pdf
file_pdf = path + "figures/Raw_data_TS_with_GCP_calibration.png"   # file name to save plot in pdf
theodo_p.plot_trajectories_prism(3, trimble_1, trimble_2, trimble_3, np.identity(4), np.identity(4), np.identity(4), plot_3d, save_pdf, file_pdf, 1)

<IPython.core.display.Javascript object>

In [8]:
## Find not moving points RTS

theodo_g = importlib.reload(theodo_g)

Groupe_index_1_list = theodo_g.list_static_points(trimble_1, limit_speed=0.01, limit_number=10)
Groupe_index_2_list = theodo_g.list_static_points(trimble_2, limit_speed=0.01, limit_number=10)
Groupe_index_3_list = theodo_g.list_static_points(trimble_3, limit_speed=0.01, limit_number=10)

In [9]:
%matplotlib notebook
plt.figure(0)
for i in Groupe_index_1_list:
	plt.scatter(trimble_1.T[i][:,0],trimble_1.T[i][:,1])
plt.show()

<IPython.core.display.Javascript object>

In [13]:
theodo_g = importlib.reload(theodo_g)

print(len(Groupe_index_1_list), len(Groupe_index_2_list), len(Groupe_index_3_list))
list_number = 20
mu_points, cov_matrix, mean_D_1_0, mean_A_1_0, mean_E_1_0, P_1_0 =theodo_g.unit_return_covariance_point(trimble_1, d1, a1, e1, Groupe_index_1_list, list_number)

num_samples = 1000
## Range
range_value = mean_D_1_0
random_noise_range = [0, 0.004/2, 2]   ## Mean, sigma, ppm,  4mm + 2ppm (2 sigma)  ISO17123-3
## Angles
true_azimuth = mean_A_1_0
true_elevation = mean_E_1_0
random_noise_angle = [0, 0.000024241/5*4/2]    # Mean, sigma, 5"=0.000024241 précison datasheet  (2 sigma)  ISO17123-3
## Tilt compensator
random_noise_tilt = [0, 0.000002424/2]    # Mean, sigma, 0.5"=0.000002424 précison datasheet  (2 sigma)  ISO17123-3

mu_raw_data, mu_points_simulated,cov_matrix_simulated = theodo_g.MC_raw_data(num_samples, range_value, random_noise_range, true_azimuth, true_elevation, random_noise_angle, random_noise_tilt, np.identity(4), [], [], 0, [0,0])

32 34 45


In [37]:
%matplotlib notebook
import matplotlib.pyplot as plt
theodo_g = importlib.reload(theodo_g)

# Setup the plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plt.xlabel('x')
plt.ylabel('y')

mu1=mu_points
cov1 = cov_matrix
s1 = np.random.multivariate_normal(mu1, cov1, size=10)
nstd = 2    # 95% confidence interval with 2
mu1_ = np.mean(s1, axis=0)
cov1_ = np.cov(s1.T)
X1,Y1,Z1 = theodo_g.get_cov_ellipsoid_bis(cov1_, mu1_, nstd)
ax.plot_wireframe(X1,Y1,Z1, color='r', alpha=0.1)
ax.scatter3D(P_1_0.T[0,:],P_1_0.T[1,:],P_1_0.T[2,:], color = 'r')

# mu2=mu_raw_data[0:3]  #mu_points_simulated
# cov2 = cov_matrix_simulated
# s2 = np.random.multivariate_normal(mu2, cov2, (200))
#
# mu2_ = np.mean(s2, axis=0)
# cov2_ = np.cov(s2.T)
# X2,Y2,Z2 = theodo_g.get_cov_ellipsoid(cov2_, mu2_, nstd)
# ax.plot_wireframe(X2,Y2,Z2, color='green', alpha=0.1)

plt.show()

/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)
/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)


<IPython.core.display.Javascript object>

In [454]:
theodo_g = importlib.reload(theodo_g)

trajectory_list_1 = theodo_g.return_uncertainty_of_discrete_trajectory(trimble_1, d1, a1, e1, Groupe_index_1_list)
trajectory_list_2 = theodo_g.return_uncertainty_of_discrete_trajectory(trimble_2, d2, a2, e2, Groupe_index_2_list)
trajectory_list_3 = theodo_g.return_uncertainty_of_discrete_trajectory(trimble_3, d3, a3, e3, Groupe_index_3_list)

path_file_GCP = "total_stations/GCP.txt"
file_name = path+path_file_GCP
_, _, _, T_1, T_2, T_3 = theodo_u.read_marker_file(file_name, 1, 1)

/home/maxime/repos/RTS_Extrinsic_Calibration/contrib/../scripts/groundtruth_utils.py:533: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [457]:
print(T_1@trajectory_list_1[0,0])
print(T_2@trajectory_list_2[0,0])
print(T_3@trajectory_list_3[0,0])

[ -1.93394025 -10.96382672   0.17808947   1.        ]
[ -2.24901602 -10.43502922  -0.22902183   1.        ]
[ -2.65064835 -11.3845699   -0.1757609    1.        ]


In [496]:
file_sensor_positions = theodo_u.if_file_exist(path + "sensors_extrinsic_calibration/sensor_positions.csv",'')
sensor_positions_list = theodo_u.read_extrinsic_calibration_results_file(file_sensor_positions)

# Load sensor positions
file_sensor_positions = theodo_u.if_file_exist(path + "sensors_extrinsic_calibration/sensor_positions.csv",'')
sensor_positions_list = theodo_u.read_extrinsic_calibration_results_file(file_sensor_positions)

Sensors = []
P1_position_RTS = np.array(sensor_positions_list[0])
P2_position_RTS = np.array(sensor_positions_list[1])
P3_position_RTS = np.array(sensor_positions_list[2])

Sensors.append(np.array(sensor_positions_list[3]))
Sensors.append(np.array(sensor_positions_list[4]))
Sensors.append(np.array(sensor_positions_list[5]))

P1_position_GNSS = P1_position_RTS - Sensors[1-1]
P1_position_GNSS[3] = 1
P2_position_GNSS = P2_position_RTS - Sensors[1-1]
P2_position_GNSS[3] = 1
P3_position_GNSS = P3_position_RTS - Sensors[1-1]
P3_position_GNSS[3] = 1

P_sensor = np.array([P1_position_GNSS,
			P2_position_GNSS,
			P3_position_GNSS]).T

In [538]:
theodo_g = importlib.reload(theodo_g)

num_samples = 100
number = 10
Pose_sensor = []
p1_corrected = theodo_g.return_point_from_covariance(trajectory_list_1[number,1], T_1@trajectory_list_1[number,0], num_samples)
p2_corrected = theodo_g.return_point_from_covariance(trajectory_list_2[number,1], T_2@trajectory_list_2[number,0], num_samples)
p3_corrected = theodo_g.return_point_from_covariance(trajectory_list_3[number,1], T_3@trajectory_list_3[number,0], num_samples)
for i,j,k in zip(p1_corrected,p2_corrected,p3_corrected):
	Q = np.array([i, j, k]).T
	Q =np.concatenate((Q, np.array([[1,1,1]])), axis=0)
	T = theodo_u.point_to_point_minimization(P_sensor, Q)
	Pose_sensor.append(T)

In [539]:
theodo_g = importlib.reload(theodo_g)
%matplotlib notebook
import matplotlib.pyplot as plt

p_T, mu_T, cov_T = theodo_g.find_noise_list_tf(Pose_sensor)

# Setup the plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plt.xlabel('x')
plt.ylabel('y')

mu1=mu_T
cov1 = cov_T
s1 = np.random.multivariate_normal(mu1, cov1, (200))

nstd = 2    # 95% confidence interval with 2
mu1_ = np.mean(s1, axis=0)
cov1_ = np.cov(s1.T)
X1,Y1,Z1 = theodo_g.get_cov_ellipsoid(cov1_, mu1_, nstd)
ax.plot_wireframe(X1,Y1,Z1, color='r', alpha=0.1)
ax.scatter3D(p_T[0,:],p_T[1,:],p_T[2,:], color = 'r')

plt.show()

/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)
/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)


<IPython.core.display.Javascript object>

# Process raw data


In [14]:
%matplotlib notebook
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)

# Important: set path to deployment data
path = "../data/20221103-2/"

# Important: change path to rosbag
file = "/home/maxime/data/FR2023/20221103/20221103_2_inter_prism.bag"

_, _, _, _, _, _, distance_1, distance_2, distance_3, azimuth_1, azimuth_2, azimuth_3, elevation_1, elevation_2, elevation_3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data(file)

/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)
/home/maxime/Python-env/RTS_python/lib/python3.8/site-packages/IPython/core/pylabtools.py:359: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.switch_backend(backend)


Number of data for theodolites: [4896 4873 4962]
Bad measures: 1444


In [15]:
theodo_u = importlib.reload(theodo_u)

Groupe_dist_1_list, Mean_list_1, index_list_1 = theodo_u.split_static_distance(distance_1, 0.01, 10)
Groupe_dist_2_list, Mean_list_2, index_list_2 = theodo_u.split_static_distance(distance_2, 0.01, 10)
Groupe_dist_3_list, Mean_list_3, index_list_3 = theodo_u.split_static_distance(distance_3, 0.01, 10)

Groupe_azim_1_list = theodo_u.split_angle_form_index_list(azimuth_1, index_list_1)
Groupe_azim_2_list = theodo_u.split_angle_form_index_list(azimuth_2, index_list_2)
Groupe_azim_3_list = theodo_u.split_angle_form_index_list(azimuth_3, index_list_3)

Groupe_elev_1_list = theodo_u.split_angle_form_index_list(elevation_1, index_list_1)
Groupe_elev_2_list = theodo_u.split_angle_form_index_list(elevation_2, index_list_2)
Groupe_elev_3_list = theodo_u.split_angle_form_index_list(elevation_3, index_list_3)

In [16]:
xs1 = range(len(Mean_list_1))
labels1 = Mean_list_1
def format_fn1(tick_val, tick_pos):
    if int(tick_val) in xs1:
        return labels1[int(tick_val)]
    else:
        return ''

xs2 = range(len(Mean_list_2))
labels2 = Mean_list_2
def format_fn2(tick_val, tick_pos):
    if int(tick_val) in xs2:
        return labels2[int(tick_val)]
    else:
        return ''

xs3 = range(len(Mean_list_3))
labels3 = Mean_list_3
def format_fn3(tick_val, tick_pos):
    if int(tick_val) in xs3:
        return labels3[int(tick_val)]
    else:
        return ''

In [17]:
%matplotlib notebook
from matplotlib.ticker import MaxNLocator

fig, axs = plt.subplots(3, 3, layout=None, figsize=(12, 8))

bp = axs[0,0].boxplot(Groupe_dist_1_list, showfliers=False)
axs[0,0].xaxis.set_major_formatter(format_fn1)
axs[0,0].xaxis.set_major_locator(MaxNLocator(integer=True))
#axs[0,0].set_xlabel('Distance  [m]')
axs[0,0].set_ylabel('Distance [mm]')
axs[0,0].set_ylim([-5,5])

bp = axs[1,0].boxplot(Groupe_azim_1_list, showfliers=False)
axs[1,0].xaxis.set_major_formatter(format_fn1)
axs[1,0].xaxis.set_major_locator(MaxNLocator(integer=True))
#axs[1,0].set_xlabel('Distance [m]')
axs[1,0].set_ylabel('Azimuth [rad]')
axs[1,0].set_ylim([-2.2e-5,2e-5])

bp = axs[2,0].boxplot(Groupe_elev_1_list, showfliers=False)
axs[2,0].xaxis.set_major_formatter(format_fn1)
axs[2,0].xaxis.set_major_locator(MaxNLocator(integer=True))
axs[2,0].set_xlabel('Distance to prism 1 [m]')
axs[2,0].set_ylabel('Elevation [rad]')
axs[2,0].set_ylim([-4.5e-5,4.5e-5])
######################################################################
bp = axs[0,1].boxplot(Groupe_dist_2_list, showfliers=False)
axs[0,1].xaxis.set_major_formatter(format_fn2)
axs[0,1].xaxis.set_major_locator(MaxNLocator(integer=True))
#axs[0,1].set_xlabel('Distance [m]')
#axs[0,1].set_ylabel('Error static \ndistance [mm]')
axs[0,1].set_ylim([-5,5])

bp = axs[1,1].boxplot(Groupe_azim_2_list, showfliers=False)
axs[1,1].xaxis.set_major_formatter(format_fn2)
axs[1,1].xaxis.set_major_locator(MaxNLocator(integer=True))
#axs[1,1].set_xlabel('Distance [m]')
#axs[1,1].set_ylabel('Error static \nazimuth [rad]')
axs[1,1].set_ylim([-2.2e-5,2e-5])

bp = axs[2,1].boxplot(Groupe_elev_2_list, showfliers=False)
axs[2,1].xaxis.set_major_formatter(format_fn2)
axs[2,1].xaxis.set_major_locator(MaxNLocator(integer=True))
axs[2,1].set_xlabel('Distance to prism 2 [m]')
#axs[2,1].set_ylabel('Error static \nelevation [rad]')
axs[2,1].set_ylim([-4.5e-5,4.5e-5])
######################################################################
bp = axs[0,2].boxplot(Groupe_dist_3_list, showfliers=False)
axs[0,2].xaxis.set_major_formatter(format_fn3)
axs[0,2].xaxis.set_major_locator(MaxNLocator(integer=True))
#axs[0,2].set_xlabel('Distance [m]')
#axs[0,2].set_ylabel('Error static \ndistance [mm]')
axs[0,2].set_ylim([-5,5])

bp = axs[1,2].boxplot(Groupe_azim_3_list, showfliers=False)
axs[1,2].xaxis.set_major_formatter(format_fn3)
axs[1,2].xaxis.set_major_locator(MaxNLocator(integer=True))
#axs[1,2].set_xlabel('Distance [m]')
#axs[1,2].set_ylabel('Error static \nazimuth [rad]')
axs[1,2].set_ylim([-2.2e-5,2e-5])

bp = axs[2,2].boxplot(Groupe_elev_3_list, showfliers=False)
axs[2,2].xaxis.set_major_formatter(format_fn3)
axs[2,2].xaxis.set_major_locator(MaxNLocator(integer=True))
axs[2,2].set_xlabel('Distance to prism 3 [m]')
#axs[2,2].set_ylabel('Error static \nelevation [rad]')
axs[2,2].set_ylim([-4.5e-5,4.5e-5])

plt.tight_layout()
plt.savefig(path + "figures/Uncertainty_static_position.jpg")
# show plot
plt.show()

<IPython.core.display.Javascript object>